In [ ]:
%load_ext micropython_magic
%reload_ext micropython_magic

In [ ]:
%mpy -s {"/dev/cu.usbmodem2101"}

In [ ]:
# %%micropython

import time
from config import config
from machine import Pin, RTC
from mp_libs.network import Network
from mp_libs.protocols import espnow_protocol
from mp_libs.protocols import min_iot_protocol
from mp_libs.protocols.wifi_protocols import WifiProtocol
from mp_libs.time import ptp

NUM_SYNC_CYCLES = 15
TIMEOUT_MSEC = 5000
TARGET_TS = 795074941607459 + 360_000_000  # 11:09pm

rtc = RTC()
miniot_network = Network.create_min_iot("GTY")
epn = miniot_network.transport.transport._transport
miniot_network.connect()

print(f"Initial RTC datetime: {rtc.datetime()}")
print(f"Initial RTC now: {rtc.now()}")
print(f"My MAC: {epn.wifi._sta.config('mac')}")
print(f"EPN Peer: {config['epn_peer_mac']}")
print(f"Initial channel: {epn.wifi._sta.config('channel')}")

# Perform espnow scan to set the right channel
miniot_network.scan()
print(f"Final channel: {epn.wifi._sta.config('channel')}")
print(f"EPN timeout: {epn.epn.config('timeout_ms')}")

target = TARGET_TS
while True:
    # Perform PTP cycle
    timestamps = ptp.sequence_periph(
        miniot_network.send,
        miniot_network.receive,
        lambda miniot_msg: miniot_msg.msg,
        TIMEOUT_MSEC,
        initiate_sync=True,
        num_sync_cycles=NUM_SYNC_CYCLES)

    # Calculate offsets
    offsets = [ptp.calculate_offset(t1, t2, t3, t4) for t1, t2, t3, t4 in timestamps]

    # print(f"timestamps: {timestamps}")
    # print(f"offsets: {offsets}")

    # Process offsets
    ave_offset = ptp.process_offsets(offsets)
    # print(f"ave_offset: {ave_offset}")

    # Update time
    # print(f"RTC before: {rtc.now()}")
    rtc.offset(ave_offset)
    # print(f"RTC after: {rtc.now()}")
    # print(f"now: {rtc.datetime()}")

    target += 15_000_000
    while rtc.now() < target:
        pass

    print(f"{rtc.now()}")
    time.sleep(1)
